## Introduction to the Data

In predicting stock prices you collect data over some period of time - day, week, month, etc. But you cannot take advantage of data from a time period until the next increment of the time period. For example, assume you collect data daily. When Monday is over you have all of the data for that day. However you can invest on Monday, because you don't get the data until the end of the day. You can use the data from Monday to invest on Tuesday.

In our research each record (row) is data for a week. Each record also has the percentage of return that stock has in the following week (percent_change_next_weeks_price). Ideally, you want to determine which stock will produce the greatest rate of return in the following week. This can help you train and test your algorithm.



### Data Dictionary
- quarter: the yearly quarter (1 = Jan-Mar; 2 = Apr=Jun).
- stock: the stock symbol (see above)
- date: the last business day of the work (this is typically a Friday)
- open: the price of the stock at the beginning of the week
- high: the highest price of the stock during the week
- low: the lowest price of the stock during the week
- close: the price of the stock at the end of the week
- volume: the number of shares of stock that traded hands in the week
- percent_change_price: the percentage change in price throughout the week
- percent_chagne_volume_over_last_wek: the percentage change in the number of shares of
- stock that traded hands for this week compared to the previous week
- previous_weeks_volume: the number of shares of stock that traded hands in the previous week
- next_weeks_open: the opening price of the stock in the following week
- next_weeks_close: the closing price of the stock in the following week
- percent_change_next_weeks_price: the percentage change in price of the stock in the
- following week days_to_next_dividend: the number of days until the next dividend
- percent_return_next_dividend: the percentage of return on the next dividend




In this project, I'll:

- Prepare the data for machine learning
- Use PCA Model, an unsupervised learning algorithm for selecting relevant features
- Train a model using Regressor model
- Measure the accuracy of the model using rmse and mse



Of course what I want to do is maximize the percent_change_next_weeks_price.

In [109]:
import pandas as pd
dows_jones = pd.read_csv("dow_jones_index.csv")
dows_jones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750 entries, 0 to 749
Data columns (total 16 columns):
quarter                               750 non-null int64
stock                                 750 non-null object
date                                  750 non-null object
open                                  750 non-null object
high                                  750 non-null object
low                                   750 non-null object
close                                 750 non-null object
volume                                750 non-null int64
percent_change_price                  750 non-null float64
percent_change_volume_over_last_wk    720 non-null float64
previous_weeks_volume                 720 non-null float64
next_weeks_open                       750 non-null object
next_weeks_close                      750 non-null object
percent_change_next_weeks_price       750 non-null float64
days_to_next_dividend                 750 non-null int64
percent_return_next_divi

In [110]:
dows_jones

,quarter,stock,date,open,high,low,close,volume,percent_change_price,percent_change_volume_over_last_wk,previous_weeks_volume,next_weeks_open,next_weeks_close,percent_change_next_weeks_price,days_to_next_dividend,percent_return_next_dividend
0,1,AA,1/7/2011,$15.82,$16.72,$15.78,$16.42,239655616,3.792670,NaN,NaN,$16.71,$15.97,-4.428490,26,0.182704
1,1,AA,1/14/2011,$16.71,$16.71,$15.64,$15.97,242963398,-4.428490,1.380223,239655616.0,$16.19,$15.79,-2.470660,19,0.187852
2,1,AA,1/21/2011,$16.19,$16.38,$15.60,$15.79,138428495,-2.470660,-43.024959,242963398.0,$15.87,$16.13,1.638310,12,0.189994
3,1,AA,1/28/2011,$15.87,$16.63,$15.82,$16.13,151379173,1.638310,9.355500,138428495.0,$16.18,$17.14,5.933250,5,0.185989
4,1,AA,2/4/2011,$16.18,$17.39,$16.18,$17.14,154387761,5.933250,1.987452,151379173.0,$17.33,$17.37,0.230814,97,0.175029
5,1,AA,2/11/2011,$17.33,$17.48,$16.97,$17.37,114691279,0.230814,-25.712195,154387761.0,$17.39,$17.28,-0.632547,90,0.172712
6,1,AA,2/18/2011,$17.39,$17.68,$17.28,$17.28,80023895,-0.632547,-30.226696,114691279.0,$16.98,$16.68,-1.766780,83,0.173611
7,1,AA,2/25/2011,$16.98,$17.15,$15.96,$16.68,132981863,-1.766780,66.177694,80023895.0,$16.81,$16.58,-1.368230,76,0.179856
8,1,AA,3/4/2011,$16.81,$16.94,$16.13,$16.58,109493077,-1.368230,-17.663150,132981863.0,$16.58,$16.03,-3.317250,69,0.180941
9,1,AA,3/11/2011,$16.58,$16.75,$15.42,$16.03,114332562,-3.317250,4.419900,109493077.0,$15.95,$16.11,1.003130,62,0.187149


## Finding Missing Values


In [111]:
dows_jones.isnull().sum()

quarter                                0
stock                                  0
date                                   0
open                                   0
high                                   0
low                                    0
close                                  0
volume                                 0
percent_change_price                   0
percent_change_volume_over_last_wk    30
previous_weeks_volume                 30
next_weeks_open                        0
next_weeks_close                       0
percent_change_next_weeks_price        0
days_to_next_dividend                  0
percent_return_next_dividend           0
dtype: int64

only 4% of percent_change_volume_over_last_wk & previous_weeks_volume are missing:
- so if I drop 4% of the entire data points (each row), it will not have so much effect on the model's outcome

In [112]:
dows_jones = dows_jones.dropna(axis=0)

In [113]:
dows_jones.notnull().sum()

quarter                               720
stock                                 720
date                                  720
open                                  720
high                                  720
low                                   720
close                                 720
volume                                720
percent_change_price                  720
percent_change_volume_over_last_wk    720
previous_weeks_volume                 720
next_weeks_open                       720
next_weeks_close                      720
percent_change_next_weeks_price       720
days_to_next_dividend                 720
percent_return_next_dividend          720
dtype: int64

## Data Cleaning and Feature Engineering

In [114]:
dows_jones["quarter"].value_counts()

2    390
1    330
Name: quarter, dtype: int64

The quarter column is a categorical feature, but it is given as a int data type

In [115]:
dows_jones["quarter"] = dows_jones.loc[:,"quarter"].astype("object")

In [116]:
dows_jones.columns

Index(['quarter', 'stock', 'date', 'open', 'high', 'low', 'close', 'volume',
       'percent_change_price', 'percent_change_volume_over_last_wk',
       'previous_weeks_volume', 'next_weeks_open', 'next_weeks_close',
       'percent_change_next_weeks_price', 'days_to_next_dividend',
       'percent_return_next_dividend'],
      dtype='object')

In [117]:
dows_jones[['open', 'high', 'low', 'close','next_weeks_open', 'next_weeks_close']]

,open,high,low,close,next_weeks_open,next_weeks_close
1,$16.71,$16.71,$15.64,$15.97,$16.19,$15.79
2,$16.19,$16.38,$15.60,$15.79,$15.87,$16.13
3,$15.87,$16.63,$15.82,$16.13,$16.18,$17.14
4,$16.18,$17.39,$16.18,$17.14,$17.33,$17.37
5,$17.33,$17.48,$16.97,$17.37,$17.39,$17.28
6,$17.39,$17.68,$17.28,$17.28,$16.98,$16.68
7,$16.98,$17.15,$15.96,$16.68,$16.81,$16.58
8,$16.81,$16.94,$16.13,$16.58,$16.58,$16.03
9,$16.58,$16.75,$15.42,$16.03,$15.95,$16.11
10,$15.95,$16.33,$15.43,$16.11,$16.38,$17.09


- These columns have the '$' signs and need to be cleaned
- They are also in a object data type and need to be converted to a float data type

In [118]:
pd.set_option('mode.chained_assignment', None)

In [119]:
dows_jones["open"] = dows_jones.loc[:,"open"].str.replace("$", " ")
dows_jones["high"] = dows_jones.loc[:,"high"].str.replace("$", " ")
dows_jones["low"] = dows_jones.loc[:,"low"].str.replace("$", " ")
dows_jones["close"] = dows_jones.loc[:,"close"].str.replace("$", " ")
dows_jones["next_weeks_close"] = dows_jones.loc[:,"next_weeks_close"].str.replace("$", " ")
dows_jones["next_weeks_open"] = dows_jones.loc[:,"next_weeks_open"].str.replace("$", " ")

In [120]:
dows_jones["low"] = dows_jones.loc[:,"low"].astype("float")
dows_jones["open"] = dows_jones.loc[:,"open"].astype("float")
dows_jones["high"] = dows_jones.loc[:,"high"].astype("float")
dows_jones["close"] = dows_jones.loc[:,"close"].astype("float")
dows_jones["next_weeks_close"] = dows_jones.loc[:,"next_weeks_close"].astype("float")
dows_jones["next_weeks_open"] = dows_jones.loc[:,"next_weeks_open"].astype("float")

In [121]:
dows_jones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 720 entries, 1 to 749
Data columns (total 16 columns):
quarter                               720 non-null object
stock                                 720 non-null object
date                                  720 non-null object
open                                  720 non-null float64
high                                  720 non-null float64
low                                   720 non-null float64
close                                 720 non-null float64
volume                                720 non-null int64
percent_change_price                  720 non-null float64
percent_change_volume_over_last_wk    720 non-null float64
previous_weeks_volume                 720 non-null float64
next_weeks_open                       720 non-null float64
next_weeks_close                      720 non-null float64
percent_change_next_weeks_price       720 non-null float64
days_to_next_dividend                 720 non-null int64
percent_return_ne

### Creating Dummies

In [122]:
cat_columns = ["quarter","stock", "date"]
dummy_df = pd.get_dummies(dows_jones[cat_columns])
dows_jones = pd.concat([dows_jones, dummy_df], axis=1)
dows_jones = dows_jones.drop(cat_columns, axis=1)

In [123]:
dows_jones

,open,high,low,close,volume,percent_change_price,percent_change_volume_over_last_wk,previous_weeks_volume,next_weeks_open,next_weeks_close,...,date_4/29/2011,date_4/8/2011,date_5/13/2011,date_5/20/2011,date_5/27/2011,date_5/6/2011,date_6/10/2011,date_6/17/2011,date_6/24/2011,date_6/3/2011
1,16.71,16.71,15.64,15.97,242963398,-4.428490,1.380223,239655616.0,16.19,15.79,...,0,0,0,0,0,0,0,0,0,0
2,16.19,16.38,15.60,15.79,138428495,-2.470660,-43.024959,242963398.0,15.87,16.13,...,0,0,0,0,0,0,0,0,0,0
3,15.87,16.63,15.82,16.13,151379173,1.638310,9.355500,138428495.0,16.18,17.14,...,0,0,0,0,0,0,0,0,0,0
4,16.18,17.39,16.18,17.14,154387761,5.933250,1.987452,151379173.0,17.33,17.37,...,0,0,0,0,0,0,0,0,0,0
5,17.33,17.48,16.97,17.37,114691279,0.230814,-25.712195,154387761.0,17.39,17.28,...,0,0,0,0,0,0,0,0,0,0
6,17.39,17.68,17.28,17.28,80023895,-0.632547,-30.226696,114691279.0,16.98,16.68,...,0,0,0,0,0,0,0,0,0,0
7,16.98,17.15,15.96,16.68,132981863,-1.766780,66.177694,80023895.0,16.81,16.58,...,0,0,0,0,0,0,0,0,0,0
8,16.81,16.94,16.13,16.58,109493077,-1.368230,-17.663150,132981863.0,16.58,16.03,...,0,0,0,0,0,0,0,0,0,0
9,16.58,16.75,15.42,16.03,114332562,-3.317250,4.419900,109493077.0,15.95,16.11,...,0,0,0,0,0,0,0,0,0,0
10,15.95,16.33,15.43,16.11,130374108,1.003130,14.030601,114332562.0,16.38,17.09,...,0,0,0,0,0,0,0,0,0,0


In [124]:
dows_jones.corr()["percent_change_next_weeks_price"]

open                                  0.066695
high                                  0.069589
low                                   0.067255
close                                 0.067789
volume                               -0.091106
percent_change_price                  0.020639
percent_change_volume_over_last_wk    0.013181
previous_weeks_volume                -0.102408
next_weeks_open                       0.067147
next_weeks_close                      0.106578
percent_change_next_weeks_price       1.000000
days_to_next_dividend                -0.028797
percent_return_next_dividend          0.103009
quarter_1                            -0.000098
quarter_2                             0.000098
stock_AA                             -0.012772
stock_AXP                             0.026857
stock_BA                             -0.000470
stock_BAC                            -0.099273
stock_CAT                             0.022296
stock_CSCO                           -0.092344
stock_CVX    

Because most of the features are not strongly correlated with the target column, a decision tree machine learning model will be effective here - I'll use Random Forest Regressor Model

Let me clearly distinguish the train data from the target data

In [125]:
dows_jones_features = dows_jones.drop(["percent_change_next_weeks_price"], axis=1)

## Training a Model with a Random Forest Regressor Model

### Using a PCA Model for Relevant Feature Selection

In [126]:
from sklearn.decomposition import PCA
# keep the first 15 principal components of the data
pca = PCA(n_components=15)
# fit PCA model to the train data
pca.fit(dows_jones_features)
# transform data onto the first 15 principal components
X_pca = pca.transform(dows_jones_features)
print("Original shape: {}".format(str(dows_jones_features.shape)))
print("Reduced shape: {}".format(str(X_pca.shape)))

Original shape: (720, 68)
Reduced shape: (720, 15)


### Picking an Error Rate

I'll use mean squared error (MSE) & root mean squared error (RMSE) because am measuring the error of a model in predicting quantitative data

In [127]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.model_selection import cross_val_predict
Y = dows_jones["percent_change_next_weeks_price"]
rf = RandomForestRegressor(random_state=1)
predictions = cross_val_predict(rf, X_pca, Y, cv=10)

In [128]:
dows_jones['squared_error'] = (predictions - dows_jones["percent_change_next_weeks_price"])**(2)
mse = dows_jones['squared_error'].mean()
print("MSE: {}".format(mse))
rmse = np.sqrt(mse)
print("RMSE: {}".format(rmse))

MSE: 1.4792945296207025
RMSE: 1.2162625249594359


A small (value) of MSE **(1.47)** & RMSE **(1.21)**  implies agreement between the prediction and the reality.